Before we start, we need to make sure that we have a Kafka cluster running and a topic that produces some streaming data. For simplicity, we will use a single-node Kafka cluster and a topic named `users`. Open the `5.0 user-gen-kafka.ipynb` notebook and execute the cell. This notebook produces a user record every few seconds and put it on a Kafka topic called `users`. 

In [26]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [27]:
builder = (SparkSession.builder
           .appName("merge-cdc-streaming")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [28]:
get_ipython().run_line_magic('load_ext', 'sparksql_magic')
get_ipython().run_line_magic('config', 'SparkSql.limit=20')

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [29]:
%%sparksql
CREATE OR REPLACE TABLE default.users (
    id INT,
    name STRING,
    age INT,
    gender STRING,
    country STRING 
) USING DELTA LOCATION '/opt/workspace/data/delta_lake/merge-cdc-streaming/users';

In [30]:
df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "earliest")
      .load())

In [31]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('gender', StringType(), True),
    StructField('country', StringType(), True)])

df = df.withColumn('value', from_json(col('value').cast("STRING"), schema))

In [32]:
df = df.select(
    col('value.id').alias('id'),
    col('value.name').alias('name'),
    col('value.age').alias('age'),
    col('value.gender').alias('gender'),
    col('value.country').alias('country'))

In [33]:
def upsertToDelta(microBatchDf, batchId):
    deltaTable = DeltaTable.forPath(spark, "/opt/workspace/data/delta_lake/merge-cdc-streaming/users" )
    (deltaTable.alias("dt")
     .merge(source=microBatchDf.alias("sdf"),
          condition="sdf.id = dt.id")
     .whenMatchedUpdate(set={
         "id": "sdf.id",
         "name": "sdf.name",
         "age": "sdf.gender",
         "country": "sdf.country"
     })
     .whenNotMatchedInsert(values={
         "id": "sdf.id",
         "name": "sdf.name",
         "age": "sdf.gender",
         "country": "sdf.country"
     })
    .execute())

In [34]:
query = (df.writeStream
         .format("delta")
         .foreachBatch(upsertToDelta)
         .outputMode("update")
         .option("checkpointLocation", "/opt/workspace/data/delta_lake/merge-cdc-streaming/users/_checkpoints/")
         .start("/opt/workspace/data/delta_lake/merge-cdc-streaming/users"))

In [35]:
%%sparksql
DESCRIBE HISTORY delta.`/opt/workspace/data/delta_lake/merge-cdc-streaming/users`;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2024-02-04 18:39:29.470000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(id#1869 = id#5736)""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,6,Serializable,False,"{'numOutputRows': '175', 'numTargetBytesAdded': '2537', 'numTargetRowsInserted': '0', 'numTargetFilesAdded': '1', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '1', 'numTargetRowsMatchedUpdated': '2', 'executionTimeMs': '2726', 'numTargetRowsCopied': '173', 'rewriteTimeMs': '646', 'numTargetRowsUpdated': '2', 'numTargetRowsDeleted': '0', 'scanTimeMs': '2030', 'numSourceRows': '1', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '2537'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
6,2024-02-04 18:39:20.134000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(id#1869 = id#4971)""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,5,Serializable,False,"{'numOutputRows': '175', 'numTargetBytesAdded': '2537', 'numTargetRowsInserted': '0', 'numTargetFilesAdded': '1', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '1', 'numTargetRowsMatchedUpdated': '1', 'executionTimeMs': '2531', 'numTargetRowsCopied': '174', 'rewriteTimeMs': '549', 'numTargetRowsUpdated': '1', 'numTargetRowsDeleted': '0', 'scanTimeMs': '1949', 'numSourceRows': '1', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '2541'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
5,2024-02-04 18:39:08.867000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(id#1869 = id#4206)""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,4,Serializable,False,"{'numOutputRows': '175', 'numTargetBytesAdded': '2541', 'numTargetRowsInserted': '0', 'numTargetFilesAdded': '1', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '1', 'numTargetRowsMatchedUpdated': '2', 'executionTimeMs': '2153', 'numTargetRowsCopied': '173', 'rewriteTimeMs': '546', 'numTargetRowsUpdated': '2', 'numTargetRowsDeleted': '0', 'scanTimeMs': '1579', 'numSourceRows': '1', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '2544'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
4,2024-02-04 18:38:59.384000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(id#1869 = id#3441)""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,3,Serializable,False,"{'numOutputRows': '175', 'numTargetBytesAdded': '2544', 'numTargetRowsInserted': '0', 'numTargetFilesAdded': '1', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '1', 'numTargetRowsMatchedUpdated': '2', 'executionTimeMs': '2485', 'numTargetRowsCopied': '173', 'rewriteTimeMs': '844', 'numTargetRowsUpdated': '2', 'numTargetRowsDeleted': '0', 'scanTimeMs': '1620', 'numSourceRows': '1', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '2551'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
3,2024-02-04 18:38:53.107000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(id#1869 = id#2676)""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,2,Serializable,False,"{'numOutputRows': '175', 'numTargetBytesAdd

In [36]:
query.stop()

In [37]:
spark.stop() 